# Train the Model

In [ ]:
import sys
import pandas as pd
from sklearn.tree import DecisionTreeClassifier

training_data = pd.read_csv('data/model_training.csv', sep=',', header=None)

X_train = training_data.values[:, 1:5]
y_train = training_data.values[:, 0]

model = DecisionTreeClassifier(criterion="gini", random_state=100, max_depth=3, min_samples_leaf=5)
model.fit(X_train, y_train)

In [ ]:
class PioBundle(object):

    def __init__(self,
                 model):

        self.model = model

        
    def predict(self,
                request):

        return self.model.predict_proba(request)


    def transform_request(self, 
                          request):

        import ujson
        import numpy as np    
        request_str = request.decode('utf-8')
        request_str = request_str.strip().replace('\n', ',')
        # surround the json with '[' ']' to prepare for conversion
        request_str = '[%s]' % request_str
        request_json = ujson.loads(request_str)

        request_transformed = ([(json_line['feature0'], json_line['feature1'], json_line['feature2'], json_line['feature3']) for json_line in request_json])
        return np.array(request_transformed) 

    
    def transform_response(self,
                           response):

        import ujson
        return ujson.dumps(response.tolist())


## Pickle `pio_bundle`

In [ ]:
import dill as pickle

pio_bundle = PioBundle(model)

pio_bundle_pkl_path = 'pio_bundle.pkl'

with open(pio_bundle_pkl_path, 'wb') as fh:
    pickle.dump(pio_bundle, fh)

In [ ]:
%%bash 

ls -l pio_bundle.pkl